In [ ]:
import os
import shutil
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import NotionDBLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
integration_token = os.getenv("NOTION_INTEGRATION_TOKEN")
database_id = os.getenv("NOTION_DATABASE_ID")
chroma_dir = './chroma_data'
model_name = 'all-MiniLM-L6-v2'
collection_name = "documents_collection"

if not integration_token or not database_id:
    raise ValueError("Missing Notion API token or Database ID in environment variables.")

In [1]:
# Initialize loaders and models
loader = NotionDBLoader(integration_token=integration_token, database_id=database_id)
embedding_function = HuggingFaceEmbeddings(model_name=model_name)
model = SentenceTransformer(model_name)

/Users/cmg203/Desktop/notion-bot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/1v/f0l_v3gs35b3lwyrxkgg9btr0000gp/T/ipykernel_33235/2511591070.py:27: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name=model_name)
/Users/cmg203/Desktop/notion-bot/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `Tru

Loading documents from Notion...
Loaded 14 documents from Notion.
Splitting documents into smaller chunks...
Split into 14 chunks.
Sanitizing metadata...
Creating Chroma vectorstore...
Chroma vectorstore created and persisted successfully.
Process completed successfully.


/var/folders/1v/f0l_v3gs35b3lwyrxkgg9btr0000gp/T/ipykernel_33235/2511591070.py:80: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
# Step 1: Load documents from Notion
print("Loading documents from Notion...")
documents = loader.load()
if not documents:
    raise ValueError("No documents were loaded from Notion.")
print(f"Loaded {len(documents)} documents from Notion.")

In [ ]:
# Step 2: Split documents into chunks
print("Splitting documents into smaller chunks...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50, separators=["\n\n", ".", " "])
chunked_documents = [
    {"content": chunk, "metadata": doc.metadata}
    for doc in documents
    for chunk in text_splitter.split_text(doc.page_content)
]
if not chunked_documents:
    raise ValueError("No chunks were created from the documents.")
print(f"Split into {len(chunked_documents)} chunks.")

In [ ]:
# Step 3: Preprocess metadata
def preprocess_metadata(chunked_documents):
    sanitized_documents = []
    for chunk in chunked_documents:
        sanitized_metadata = {
            key: (value[0] if isinstance(value, list) and len(value) > 0 else value)
            if isinstance(value, list) else value
            for key, value in chunk["metadata"].items()
        }
        sanitized_documents.append({
            "content": chunk["content"],
            "metadata": sanitized_metadata
        })
    return sanitized_documents

print("Sanitizing metadata...")
sanitized_documents = preprocess_metadata(chunked_documents)

In [ ]:
# Step 4: Create and persist Chroma vectorstore
print("Creating Chroma vectorstore...")
texts = [doc["content"] for doc in sanitized_documents]
metadatas = [doc["metadata"] for doc in sanitized_documents]

try:
    vectorstore = Chroma.from_texts(
        texts=texts,
        embedding=embedding_function,
        metadatas=metadatas,
        persist_directory=chroma_dir,
        collection_name=collection_name,
    )
    vectorstore.persist()
    print("Chroma vectorstore created and persisted successfully.")
except Exception as e:
    raise RuntimeError(f"Error creating Chroma vectorstore: {e}")

print("Process completed successfully.")